In [ ]:
# Домашнее задание (Кросс-валидация)
# Преподаватель: Алексей Кузьмин

# См. материалы к занятию.

# Для выполнения домашнего задания необходимо взять boston 
# house-prices datase (sklearn.datasets.load_boston) и сделать 
# тоже самое для задачи регрессии (попробовать разные алгоритмы, 
# поподбирать параметры, вывести итоговое качество). 


In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [9]:
data = pd.DataFrame(boston['data'], columns=boston['feature_names'])
y = boston['target']

In [7]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
dtypes: float64(13)
memory usage: 51.5 KB


In [67]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [48]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
# Подберем рпзличные типы моделей и величины гиперпараметров 
dict_estimator = {}

In [39]:
from sklearn.linear_model import LinearRegression
dict_estimator['lr'] = {
    'est': LinearRegression(),
    'param': {}
}

In [40]:
from sklearn.linear_model import Ridge
dict_estimator['Ridge'] = {
    'est': Ridge(),
    'param': {'alpha':[0.001, 0.01, 0.1, 0.2, 0.5, 1]}
}

In [41]:
from sklearn.linear_model import BayesianRidge
dict_estimator['BayesR'] = {
    'est': BayesianRidge(),
    'param': {'alpha_1':[1.e-7,1.e-6,1.e-5],
              'alpha_2':[1.e-7,1.e-6,1.e-5], 
              'lambda_1':[1.e-7,1.e-6,1.e-5],
              'lambda_2':[1.e-7,1.e-6,1.e-5]}
}

In [42]:
from sklearn.linear_model import PassiveAggressiveRegressor
dict_estimator['PAR'] = {
    'est': PassiveAggressiveRegressor(),
    'param': {'tol':[None,1e-4,1e-3,1e-2],
              'loss': ['epsilon_insensitive','squared_epsilon_insensitive']}
}

In [43]:
from sklearn.tree import ExtraTreeRegressor
dict_estimator['ExtTR'] = {
    'est': ExtraTreeRegressor(),
    'param': {'max_depth':[3,5,7],
              'min_samples_split':[0.1, 0.2, 0.5,1.0]}
}

In [44]:
from sklearn.tree import DecisionTreeRegressor
dict_estimator['DTR'] = {
    'est': DecisionTreeRegressor(),
    'param': {'max_depth':[3,5,7],
              'min_samples_split':[0.1, 0.2, 0.5,1.0],
              'criterion':['mse','mae']}
}

In [45]:
from sklearn.svm import LinearSVR
dict_estimator['SVR'] = {
    'est': LinearSVR(),
    'param': {'C':[0.1,0.2,0.5,1],
              'loss':['epsilon_insensitive','squared_epsilon_insensitive']}
}

In [46]:
from sklearn.neighbors import KNeighborsRegressor
dict_estimator['KNR'] = {
    'est': KNeighborsRegressor(),
    'param': {'n_neighbors':[1,3,5,7],
              'weights':['uniform','distance']}
}

In [47]:
from sklearn.linear_model import SGDRegressor
dict_estimator['SGDR'] = {
    'est': SGDRegressor(),
    'param': {'penalty':['none','l2', 'l1', 'elasticnet'],
              'loss':['squared_loss','huber']}
}

In [127]:
# Отмасштабируем данные и разобъем на тесовую и обучающую выборки
data = StandardScaler().fit_transform(boston['data'])
X, X_t, y, y_t = train_test_split(data,boston['target'], test_size=0.3)

In [130]:
# Обучим модели, получим предсказания, посчитаем и выведем r2_score 
# 
for key in dict_estimator:
    dict_estimator[key]['clf'] = GridSearchCV(dict_estimator[key]['est'], dict_estimator[key]['param'], cv=5).fit(X,y)
    dict_estimator[key]['predict'] = dict_estimator[key]['clf'].predict(X_t)
    dict_estimator[key]['score'] = r2_score(y_t,dict_estimator[key]['predict'])
    print(key, dict_estimator[key]['score'])
    
# среднее значение r2_score    
print('\nmean:',(np.array([dict_estimator[key]['score'] for key in dict_estimator])).mean())

lr 0.7593165379630451
Ridge 0.7611971688269545
BayesR 0.767573213289828
PAR 0.5348900943760693
ExtTR 0.6164997103678858
DTR 0.8022068781824355
SVR 0.7608006143762889
KNR 0.83742187914326
SGDR 0.7805643767348032

mean: 0.7356078303622856


In [ ]:
# по поводу некоторых алгоритмов возникли вопросы
# PAR, SGDR - при различных разбиениях выборки получались отрицательные значения r2_score
#
#
#
#